# Bllossom-AICA Instruction Tuning Tutorial (Only Text)

## Library Import

In [ ]:
!pip install datasets bitsandbytes peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 34.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 92.1 MB/s eta 0:00:00
  Using cached multiprocess-0.70.16-py310-none-any.whl (134 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (194 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 52.9 MB/s eta 0:00:0000:0100:01
  Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 83.8 MB/s eta 0:00:00
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 86.1 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached frozenlist-1.5.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manyli

In [ ]:
import os
from transformers import MllamaForConditionalGeneration, AutoTokenizer
from transformers import Trainer, TrainingArguments
import torch
from torch.nn.utils.rnn import pad_sequence
import datasets
from peft import LoraConfig, get_peft_model

os.environ['CUDA_VISIBLE_DEVICES']="0"

/home/hgyoo/graaadio/.graadio/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/home/hgyoo/graaadio/.graadi

## Model & Tokenizer Load

In [ ]:
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-AICA-5B'

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map='cuda:0',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  4.85it/s]


In [ ]:
# 학습 전
sample_message = [
    {'role':'user', 'content': [
        {'type':'text', 'text':'자연어처리 교과목에 대해 간략히 소개해줘'}
        ]},
]

inputs = tokenizer.apply_chat_template(sample_message,
                              tokenize=True,
                              add_generation_prompt=True,
                              return_tensors='pt').to(model.device)


output = model.generate(inputs,max_new_tokens=512,eos_token_id=tokenizer.convert_tokens_to_ids('<|eot_id|>'))

print(tokenizer.decode(output[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 13 Dec 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

자연어처리 교과목에 대해 간략히 소개해줘<|eot_id|><|start_header_id|>assistant<|end_header_id|>

자연어처리(Natural Language Processing, NLP)는 컴퓨터가 인류의 언어를 이해하고 생성할 수 있도록 하는 기술입니다. 이 교과목은 다음과 같은 주요 주제들을 포함할 수 있습니다:

1. **문서형화(Annotation)**: 인간이 텍스트를 이해하는 개체용 데이터를 제공하는 과정. 예를 들어, 챗봇이 사용하는 고객 반응 데이터.

2. **자연어 처리(NLP)**: 컴퓨터가 인간의 언어를 이해하고 처리하는 기술. 예를 들어, 음성 인식, 텍스트 분석, 번역 등이 있습니다.

3. **언어 모델(Language Models)**: 대규모 텍스트 데이터를 바탕으로 한 언어 모델. 이 모델은 텍스트의 문맥을 이해하고, 관련된 단어를 예측하는 데 사용됩니다.

4. **언어 이해(Language Understanding)**: 텍스트가 실제로 무엇을 의미하는지 이해하는 기술. 이는 특정 질문에 대한 답변을 찾거나, 뉴스 기사에서 핵심 정보를 추출하는 작업에 사용됩니다.

5. **기계번역(Machine Translation)**: 한 언어의 텍스트를 다른 언어로 자동으로 번역하는 기술.

6. **챗봇(Chatbots)**: 대화형 컴퓨터 프로그램. 텍스트, 음성, 혹은 여러 형태의 인터랙션을 통해 사용자와 상호작용합니다.

자연어처리는 인공지능(IAI) 분야에 속하며, 인류의 언어와 텍스트 데이터를 분석하고 이해하는 기술을 개발하는 데 중점을 두고 있습니다. 이 교과목은 기계 학습, 인공 신경,

## Dataset Load

In [ ]:
train_dataset = datasets.load_dataset('beomi/KoAlpaca-v1.1a',split='train[:100]')
train_dataset

Dataset({
    features: ['instruction', 'output', 'url'],
    num_rows: 100
})

In [ ]:
train_dataset[0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

## Data Preprocessing

In [ ]:
def preprocessing_data(examples):
    input_ids = []
    attention_masks = []
    labels = []

    for instruction,response in zip(examples['instruction'],examples['output']):
        message = [
        {'role': 'user', 'content': [
            {'type':'text', 'text':instruction}
        ]},
        ]
        inputs = tokenizer.apply_chat_template(message, tokenize=True, add_generation_prompt=True)
        label = tokenizer(response+'<|eot_id|>',add_special_tokens=False)['input_ids']
        input_id = inputs+label+([tokenizer.pad_token_id]*4096)
        input_id = input_id[:4096]
        label_id = [-100]*len(inputs) + label + ([tokenizer.pad_token_id]*4096)
        label_id = label_id[:4096]
        attention_mask = [1 if token != tokenizer.pad_token_id else 0 for token in input_id]

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        labels.append(label_id)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_masks,
        'labels': labels
    }

In [ ]:
train_dataset = train_dataset.map(
    preprocessing_data,
    num_proc=2,
    batched=True,
    remove_columns=['instruction','output','url']
)

train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [ ]:
train_dataset[0]["input_ids"][:10]

[128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25]

In [ ]:
tokenizer.decode(train_dataset[0]["input_ids"][:100])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 13 Dec 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해'

## Data Collator

In [ ]:
def DataCollator(examples):
    input_ids = torch.LongTensor([example['input_ids'] for example in examples])
    attention_mask = torch.LongTensor([example['attention_mask'] for example in examples])
    labels = torch.LongTensor([example['labels'] for example in examples])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }


## Configuration TrainingArugments

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    remove_unused_columns=False,
    report_to="none",
    optim="adamw_bnb_8bit",
    gradient_checkpointing=True,
    num_train_epochs=1,
    logging_strategy='steps',
    logging_steps=10,
    label_names=['labels'],
    torch_compile=True,
)


## Configuration Trainable Parameters

In [ ]:
# LLM Full Tuning
target_modules = []
for n,p in model.named_parameters():
    if 'language_model' in n and 'cross_attn' not in n and 'embed_tokens' not in n and 'lm_head' not in n and 'norm' not in n:
        target_modules.append(n.replace('.weight',''))
    # else:
    #     p.require_grad=False

# Lora Tuning
peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=4,  #<-- 요거 줄이면 GPU메모리 절약됩니다.
    lora_alpha=8, #<-- 요거 줄이면 GPU메모리 절약됩니다.
    target_modules=target_modules
)

lora_model = get_peft_model(model,peft_config)
lora_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MllamaForConditionalGeneration(
      (vision_model): MllamaVisionModel(
        (patch_embedding): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), padding=valid, bias=False)
        (gated_positional_embedding): MllamaPrecomputedPositionEmbedding(
          (tile_embedding): Embedding(9, 8197120)
        )
        (pre_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
          (embedding): Embedding(9, 5120)
        )
        (post_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
          (embedding): Embedding(9, 5120)
        )
        (layernorm_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (layernorm_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (transformer): MllamaVisionEncoder(
          (layers): ModuleList(
            (0-31): 32 x MllamaVisionEncoderLayer(
              (self_attn): MllamaVisionSdpaAttention(
               

## Train

In [ ]:
# LLM 풀튜닝 VRAM 30GB 사용
# LoRA 사용시 18.6GB 사용됨 (RANK 16, 토큰임베딩 및 lm_head 학습 X)
# 메모리 터질 시 RANK 사이즈 등 조정해서 사용하세용~
trainer = Trainer(
        model=lora_model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=DataCollator,
    )

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,10.714800
20,7.725700
30,6.368400
40,5.944000
50,5.738000
60,5.690900
70,5.594900
80,5.615000
90,5.639300
100,5.615500


TrainOutput(global_step=100, training_loss=6.464637794494629, metrics={'train_runtime': 370.3354, 'train_samples_per_second': 0.27, 'train_steps_per_second': 0.27, 'total_flos': 1.16953090646016e+16, 'train_loss': 6.464637794494629, 'epoch': 1.0})

## Inference Test

In [ ]:
# 학습 후
sample_message = [
    {'role':'user', 'content': [
        {'type':'text', 'text':'서울의 유명 관광지에 대해 소개해줘'}
        ]},
]

inputs = tokenizer.apply_chat_template(sample_message,
                              tokenize=True,
                              add_generation_prompt=True,
                              return_tensors='pt').to(model.device)


output = lora_model.generate(inputs,max_new_tokens=512,eos_token_id=tokenizer.convert_tokens_to_ids('<|eot_id|>'))

print(tokenizer.decode(output[0]))